In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
%matplotlib inline

In [2]:
# zipfile.ZipFile('data/data36883/腾讯.zip').extractall('./work')

In [3]:
# zipfile.ZipFile('work/train_preliminary.zip').extractall('./work')
# zipfile.ZipFile('work/test.zip').extractall('./work')

In [4]:
train_path = "work/train_preliminary/"
test_path = "work/test/"

In [5]:
train_ad = pd.read_csv(train_path + "ad.csv")
train_click_log = pd.read_csv(train_path + "click_log.csv")
train_user = pd.read_csv(train_path + "user.csv")

In [6]:
test_ad = pd.read_csv(test_path + "ad.csv")
test_click_log = pd.read_csv(test_path + "click_log.csv")

In [7]:
train_ad.shape, train_click_log.shape, train_user.shape

((2481135, 6), (30082771, 4), (900000, 3))

In [8]:
test_ad.shape, test_click_log.shape

((2618159, 6), (33585512, 4))

In [9]:
train_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [10]:
train_ad.creative_id.value_counts()

2047       1
3474441    1
3505158    1
3494915    1
3492864    1
701439     1
693243     1
695290     1
4406371    1
719862     1
713717     1
709619     1
705521     1
668655     1
670702     1
664557     1
666604     1
3503111    1
3484684    1
687078     1
3482637    1
3410984    1
3439654    1
3433509    1
3435556    1
3429411    1
3431458    1
3515421    1
3517468    1
3511323    1
          ..
2122072    1
2103639    1
2101590    1
2099541    1
2107729    1
2152783    1
2148685    1
2146636    1
2158922    1
2156873    1
2154824    1
2117982    1
2120031    1
2204000    1
2171248    1
2167158    1
2165109    1
4322370    1
2177395    1
2175346    1
2173297    1
2226539    1
2206049    1
2224490    1
2220392    1
2201959    1
2197861    1
2195812    1
2210147    1
4194304    1
Name: creative_id, Length: 2481135, dtype: int64

In [11]:
train_click_log.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [12]:
train_click_log.time.value_counts()

85    417797
78    406004
84    393253
64    388689
57    384115
74    381696
91    376422
63    376263
34    373768
36    371639
31    369978
77    368169
73    367376
43    363130
35    362195
75    360971
65    360797
37    359112
32    358182
33    357740
71    357655
56    357324
79    355733
67    354459
50    353179
70    350897
66    349208
88    349030
90    348829
81    348312
       ...  
15    317617
29    316204
44    314554
13    312896
39    310426
21    309926
18    307755
28    307321
38    305780
1     298107
26    293122
17    293056
8     291980
19    291491
16    289072
25    288726
11    288523
7     287337
27    286883
23    286319
2     284214
24    283650
12    282593
10    275954
3     273636
20    273404
4     271385
5     268042
9     259223
6     256799
Name: time, Length: 91, dtype: int64

In [16]:
test_click_log.head()

,time,user_id,creative_id,click_times
0,20,3131989,645764,1
1,20,3131989,1027422,1
2,20,3131989,1106443,1
3,20,3131989,629802,1
4,59,3131989,2839769,1


In [22]:
sub = pd.DataFrame(test_click_log['user_id'])

In [25]:
sub['age'] = 1
sub['gender'] = 1

In [26]:
sub.to_csv('submission.csv')